### Stanford Dog Breed 데이터 세트를 아래 URL에서 직접 Download 및 압축 해제
* Kaggle의 Dataset으로 Object Storage 연결 시 이미지를 한장 씩 읽는 데 많은 시간이 소요되어 모델 학습에 시간이 더 걸림. 
* Local Disk에 바로 이미지를 다운로드/압축 해제 후 모델에서 이를 이용할 수 있도록 함. 

In [ ]:
# stanford dog breed 데이터 세트 다운로드 
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
# 현재 디렉토리인 /kaggle/working에 바로 압축 해제 
!ls; tar -xvf images.tar

In [ ]:
# 현재 디렉토리의 내용 확인. 
!ls; pwd

In [ ]:
!cd /kaggle/working/Images;ls

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/working/Images'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 이미지 파일들의 디렉토리와 파일명을 기반으로 메타 정보인 이미지 절대경로, 레이블을 DataFrame으로 생성
* /kaggle/working/Images 디렉토리 밑에 Dog breed 서브 디렉토리와 이미지 파일로 구성 되어 있음. 
* 레이블 값은 이미지 파일의 절대경로에서 이미지 파일 바로 위에 있는 서브 디렉토리를 가공하여 생성. 

In [ ]:
import pandas as pd
import numpy as np
import os 

IMAGE_DIR = '/kaggle/working/Images' 

def make_dogbreed_dataframe(image_dir=IMAGE_DIR):
    paths = []
    label_gubuns = []
    for dirname, _, filenames in os.walk(image_dir):
        for filename in filenames:
            # 이미지 파일이 아닌 파일도 해당 디렉토리에 있음.
            if '.jpg' in filename:
                # 파일의 절대 경로를 file_path 변수에 할당. 
                file_path = dirname+'/'+ filename
                paths.append(file_path)
                # 이미지 파일의 절대 경로에서 레이블명 생성을 위한 1차 추출. '/'로 분할하여 파일 바로 위 서브디렉토리 이름 가져옴.  
                start_pos = file_path.find('/', 20)
                end_pos = file_path.rfind('/')
                imsi_breed = file_path[start_pos+1:end_pos]
                # 1차 추출된 데이터를 기반으로 '-' 이후 데이터가 레이블 값임. 
                breed = imsi_breed[imsi_breed.find('-')+1:]
                #print(start_pos, end_pos, imsi_breed, breed)
                label_gubuns.append(breed)

    data_df = pd.DataFrame({'path':paths, 'label':label_gubuns})
    
    #label에 따른 숫자 target 값 매핑
    sorted_label = np.sort(data_df['label'].unique())
    label_mapping = {label: index for index, label in enumerate(sorted_label)}
    data_df['target'] = data_df['label'].map(label_mapping)
    data_df = data_df.sort_values(by=['target', 'path'], ascending=True)
    
    return data_df


In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 200)

data_df = make_dogbreed_dataframe()
print('data_df shape:', data_df.shape)
data_df.head()

In [ ]:
sorted_label = np.sort(data_df['label'].unique())

label_mapping = {label: index for index, label in enumerate(sorted_label)}
print(label_mapping)


### Dog Breed의 개별 분포도 확인. 

In [ ]:
print(data_df.shape)
# breed 별 건수 확인
data_df[['label', 'target']].value_counts()

In [ ]:
# 막대 그래프 형태로 breed별 건수 확인 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

plt.figure(figsize=(26, 4))

sns.countplot(data=data_df, x='label')
plt.xticks(rotation=90)
plt.show()

### Dog Breed의 이미지 보기

In [ ]:
import cv2

# dog breed 별로 image를 보기 위한 utility 함수 생성.
def show_grid_images(image_path_list, ncols=8, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        axs[i].imshow(image)
        axs[i].set_title(title)  

In [ ]:
breed_image_list_01 = data_df[data_df['label']=='Staffordshire_bullterrier']['path'].iloc[:6].tolist()
breed_image_list_02 = data_df[data_df['label']=='American_Staffordshire_terrier']['path'].iloc[:6].tolist()

show_grid_images(breed_image_list_01, ncols=6, title='Staffordshire_bullterrier')
show_grid_images(breed_image_list_02, ncols=6, title='American_Staffordshire_terrier')

In [ ]:
data_df['label'].value_counts().index.tolist()

In [ ]:
breed_list = data_df['label'].value_counts().index.tolist()

for iter_cnt, breed in enumerate(breed_list):
    breed_image_list = data_df[data_df['label']==breed]['path'].iloc[:6].tolist()
    show_grid_images(breed_image_list, ncols=6, title=breed)
    if iter_cnt == 4:
        break

### Augmentation 적용한 이미지 살펴 보기

In [ ]:
import albumentations as A

# crop은 사용에 주의할것. 사람과 개가 같이 있으므로 center에 사람이 있을 경우 사람만 이미지가 잘릴수 있음. 
imsi_augmentor = A.Compose([
    A.Resize(height=224, width=224, p=1), 
    A.CenterCrop(height=200, width=200, p=1),#A.CenterCrop(height=180, width=180, p=1)
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
    A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5)
])

# augmented 적용된 이미지를 보기 위해서 위에서 albumetation으로 image 변환 기법 적용된 transformer 입력
# 이미지 사이즈를 224x224로 resize 적용. 
def show_grid_images(image_path_list, augmentor=None, ncols=4, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        if augmentor is not None:
            image = augmentor(image=image)['image']
        axs[i].imshow(image)
        axs[i].axis('off')
        axs[i].set_title(title) 
        
breed_image_list_01 = data_df[data_df['label']=='Staffordshire_bullterrier']['path'].iloc[:6].tolist()       
show_grid_images(breed_image_list_01, augmentor=None, ncols=6, title='orignal Staffordshire_bullterrier')
show_grid_images(breed_image_list_01, augmentor=imsi_augmentor, ncols=6, title='augmented Staffordshire_bullterrier')

### 전체 DataFrame을 학습과 테스트용 DataFrame으로 분리. 학습 DataFrame은 다시 학습과 검증용으로 분리 
* train_test_split()을 이용하여 전체의 40%를 테스트 데이터로 할당. stratify인자로 breed label별로 균등하게 할당 설정. 

In [ ]:
from sklearn.model_selection import train_test_split

# 전체 데이터의 60%를 학습, 40%를 테스트로 분리. 
train_df, test_df = train_test_split(data_df, test_size=0.4, stratify=data_df['label'], random_state=2025)
print(train_df.shape, test_df.shape)

In [ ]:
print(train_df['label'].value_counts()/train_df.shape[0])
print(test_df['label'].value_counts()/test_df.shape[0])

In [ ]:
# 다시 학습 데이터의 80%를 학습, 20%를 검증으로 분리
tr_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['target'], random_state=2025)

### Custom Dataset와 DataLoader 생성
* augmentation은 light한 augmention 부터 시작하여 점차 강도를 높임

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2

class BreedDataset(Dataset):
    # 이미지 파일리스트, 타겟 파일리스트, transforms 등 이미지와 타겟 데이터 가공에 필요한 인자들을 입력 받음
    def __init__(self, image_paths, targets=None, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform
    
    # 전체 건수를 반환
    def __len__(self):
        return len(self.image_paths)
        
    # idx로 지정된 하나의 image, label을 tensor 형태로 반환
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        # opencv로 이미지 파일 로딩
        image_np = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB).astype(np.float32)
        # 보통은 transform이 None이 되는 경우는 거의 없음(Tensor 변환이라도 있음)
        image = self.transform(image=image_np)['image']

        if self.targets is not None:
            # 개별 target값을 tensor로 변환.
            target = torch.tensor(self.targets[idx])
            return image, target
        # 테스트 데이터의 경우 targets가 입력 되지 않을 수 있으므로 이를 대비. 
        else:
            return image

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

BATCH_SIZE = 16
class CFG:
    batch_size = 16
    image_size = 224

# Data Augmentation
tr_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size, p=1),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

def create_tr_val_loader(tr_df, val_df, tr_transform, val_transform):
    tr_dataset =BreedDataset(image_paths=tr_df['path'].to_list(), 
                               targets=tr_df['target'].to_list(), transform=tr_transform)
    val_dataset = BreedDataset(image_paths=val_df['path'].to_list(), 
                               targets=val_df['target'].to_list(), transform=val_transform)
    
    tr_loader = DataLoader(tr_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=4*BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    return tr_loader, val_loader

tr_loader, val_loader = create_tr_val_loader(tr_df=tr_df, val_df=val_df, 
                                             tr_transform=tr_transform, val_transform=val_transform)
images, labels = next(iter(tr_loader))
print(images.shape, labels.shape)


### torchvision Model 생성.
* Resnet101과 Efficientnet 계열을 테스트 할 수 있도록 모델 생성 함수 생성.  

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

def create_tv_model(model_name, num_classes=1000):
    model = None
    if model_name == 'efficientnet_v2_s':
        model = models.efficientnet_v2_s(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'efficientnet_b4':
        model = models.efficientnet_b4(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1792, out_features=num_classes))
    elif model_name == 'efficientnet_b1':
        model = models.efficientnet_b1(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'resnet101':
        model = models.resnet101(weights='DEFAULT')
        model.fc = nn.Linear(in_features=2048, out_features=num_classes)
        
    return model

eff_model = create_tv_model('efficientnet_b0', num_classes=120)

In [ ]:
from torchvision import models

models.resnet101(weights=None)#resnet101, efficientnet_b0, efficientnet_v2_s

In [ ]:
models.ResNet101_Weights.IMAGENET1K_V2.transforms() # EffcientNet_B0

### Trainer로 모델 학습. 
* resnet101과 efficientnet B0/B1

In [ ]:
# /kaggle/working/modular/v1 디렉토리에 utils.py 파일 다운로드
!rm -rf ./modular/v1
!mkdir -p ./modular/v1
!wget -O ./modular/v1/utils.py https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/main/modular/v1/utils.py?raw=true
!ls ./modular/v1

import sys

# 반드시 system path를 아래와 같이 잡아줘야 함. 
sys.path.append('/kaggle/working')

#아래가 수행되는지 반드시 확인
from modular.v1.utils import Trainer, ModelCheckpoint, EarlyStopping

In [ ]:
from torch.optim import Adam
import torch.optim as optim
import torchmetrics
import timm

CFG.batch_size = 32 # 16
CFG.image_size = 224

# Horizontal_flip
tr_transform_01 = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size, p=1),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

#learning_rate, callbacks, epochs는 인자로 입력
def train_breed(model_name, tr_transform, val_transform, learning_rate=1e-3, callbacks=None, epochs=30):
    tr_loader, val_loader = create_tr_val_loader(tr_df=tr_df, val_df=val_df, 
                                             tr_transform=tr_transform, val_transform=val_transform)
    model = create_tv_model(model_name, num_classes=120)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    optimizer = Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer=optimizer, mode='min', factor=0.2, patience=3, threshold=0.01, min_lr=1e-7)
    
    trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
                   train_loader=tr_loader, val_loader=val_loader, scheduler=scheduler, 
                   callbacks=callbacks, device=device)
    # 학습 및 평가. epochs는 20회로 조정. 
    history = trainer.fit(epochs)
    
    return trainer, history

In [ ]:
#모델은 resnet101, HorizontalFlip만 augmentation적용. learning_rate는 1e-4, callbacks는 None, epochs는 30회 수행. 
trainer, history = train_breed('resnet101', tr_transform_01, val_transform, learning_rate=1e-4, 
                               callbacks=None, epochs=30)

In [ ]:
#아래가 수행되는지 반드시 확인
from modular.v1.utils import Predictor

test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

#CFG.batch_size = 16
CFG.image_size = 224

test_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_dataset = BreedDataset(image_paths=test_image_paths, 
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

#### 어떤 Breed가 예측이 많이 틀렸는지 확인 
* 실제 Ground Truth Breed와 예측 Breed가 어떻게 틀려졌는지 확인. 

In [ ]:
# 모든 테스트 데이터에 대해서 예측 수행 후 numpy로 반환 
def get_all_predictions(predictor, test_loader):
    preds_all_list = []
    targets_all_list = []
    
    for images, targets in test_loader:
        preds = predictor.predict(images).cpu().numpy()
        # 개별 원소값으로 저장해야 하므로 append()가 아니라 extend()로 list에 저장. 
        preds_all_list.extend(preds)
        # targets_all_list.extend(targets.cpu().numpy())

    preds_all = np.array(preds_all_list)

    return preds_all

preds_all = get_all_predictions(predictor, test_loader)

#예측 결과를 test_df의 별도 컬럼으로 저장. 
test_df['resnet101_pred'] = preds_all

In [ ]:
test_df.head()

In [ ]:
# 예측이 틀린 데이터 확인 
test_df[test_df['target'] != test_df['resnet101_pred']]

In [ ]:
test_df[test_df['target'] != test_df['resnet101_pred']]['label'].value_counts()

In [ ]:
test_df[test_df['label'] == 'Eskimo_dog'][['target', 'resnet101_pred']].value_counts()

In [ ]:
sorted_label = np.sort(data_df['label'].unique())
#print(sorted_label)
print(sorted_label[[24, 64, 101]])

In [ ]:
def show_grid_images(image_path_list, augmentor=None, ncols=4, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        if augmentor is not None:
            image = augmentor(image=image)['image']
        axs[i].imshow(image)
        axs[i].axis('off')
        axs[i].set_title(title) 
        
breed_image_list_01 = data_df[data_df['label']=='Siberian_husky']['path'].iloc[:6].tolist()
breed_image_list_02 = data_df[data_df['label']=='Eskimo_dog']['path'].iloc[:6].tolist()

show_grid_images(breed_image_list_01, ncols=6, title='Siberian_husky')
show_grid_images(breed_image_list_02, ncols=6, title='Eskimo_dog')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

wrong_result_df = test_df[test_df['target'] != test_df['resnet101_pred']]

plt.figure(figsize=(26, 4))
plt.xticks(rotation=90)
sns.countplot(data=wrong_result_df, x='label')

#### efficient b0로 모델 학습/평가
* 동일한 batch size, image size, augmentation으로 efficientnet_b0 모델 학습 및 평가 

In [ ]:
models.EfficientNet_B0_Weights.IMAGENET1K_V1.transforms() # EffcientNet_B0

In [ ]:
from torch.optim import Adam
import torch.optim as optim
import torchmetrics
import timm

# 동일한 batch size, image size, augmentation으로 efficientnet_b0 모델 학습 및 평가. 
trainer, history = train_breed('efficientnet_b0', tr_transform_01, val_transform, learning_rate=1e-4, 
                               callbacks=None, epochs=30)

In [ ]:
trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

#### EfficientNet B1 으로 학습/평가
* image size를 240으로 설정

In [ ]:
models.EfficientNet_B1_Weights.IMAGENET1K_V2.transforms() # EffcientNet_B0

In [ ]:
from torch.optim import Adam
import torch.optim as optim
import torchmetrics
import timm

CFG.batch_size = 32 # 16
CFG.image_size = 240

# Horizontal_flip
tr_transform_01 = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size, p=1),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

trainer, history = train_breed('efficientnet_b1', tr_transform_01, val_transform, learning_rate=1e-4, 
                               callbacks=None, epochs=30)

In [ ]:
#아래가 수행되는지 반드시 확인
from modular.v1.utils import Predictor

# 테스트 이미지 사이즈를 240으로 증가. 
CFG.image_size = 240

test_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_dataset = BreedDataset(image_paths=test_image_paths, 
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

### 다양한 Augmentation을 적용하여 모델 학습 및 성능 평가
* Heavy한 Augmentation을 적용한다고 반드시 성능이 향상되는 것이 아님(오히려 성능이 저하될 수 있음)

In [ ]:
from torch.optim import Adam
import torch.optim as optim

CFG.batch_size = 32 # 16
CFG.image_size = 240

# Horizontal_flip
from torch.optim import Adam
import torch.optim as optim
import torchmetrics
import timm

CFG.batch_size = 32 # 16
CFG.image_size = 240

tr_transform_02 = A.Compose([
    A.RandomResizedCrop(height=180, width=180, scale=(0.5, 1.0), p=0.3),
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.3),
    A.ShiftScaleRotate(p=0.2),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.2),
    A.ColorJitter(p=0.2),
    A.OneOf(
        [A.CoarseDropout(p=1, max_holes=26), 
         A.CLAHE(p=1)
        ], p=0.3), 
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [ ]:
breed_image_list_01 = data_df[data_df['label']=='Staffordshire_bullterrier']['path'].iloc[:6].tolist()       
show_grid_images(breed_image_list_01, augmentor=None, ncols=6, title='orignal Staffordshire_bullterrier')
show_grid_images(breed_image_list_01, augmentor=tr_transform_02, ncols=6, title='augmented')

In [ ]:
trainer, history = train_breed('efficientnet_b1', tr_transform_02, val_transform, learning_rate=1e-4, epochs=30)

In [ ]:
#아래가 수행되는지 반드시 확인
from modular.v1.utils import Predictor

test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

# 테스트 이미지 사이즈를 240으로 증가. 
CFG.image_size = 240

test_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_dataset = BreedDataset(image_paths=test_image_paths, 
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

#### 다른 Augumentation 적용 후 모델 학습 평가

In [ ]:
tr_transform_03 = A.Compose([
    A.HorizontalFlip(p=0.3),
    A.ShiftScaleRotate(scale_limit=(0.7, 0.9), p=0.2, rotate_limit=30),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.2),
    A.ColorJitter(p=0.2),
    A.Resize(CFG.image_size, CFG.image_size, p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [ ]:
breed_image_list_01 = data_df[data_df['label']=='Staffordshire_bullterrier']['path'].iloc[:6].tolist()       
show_grid_images(breed_image_list_01, augmentor=None, ncols=6, title='orignal Staffordshire_bullterrier')
show_grid_images(breed_image_list_01, augmentor=tr_transform_03, ncols=6, title='augmented')

In [ ]:
trainer, history = train_breed('efficientnet_b1', tr_transform_03, val_transform, learning_rate=1e-4, epochs=30)

In [ ]:
trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')